# Basic Usage - Topographic Classification

Tutorial dasar penggunaan package topoclassify untuk klasifikasi topografi 24 kelas

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from topoclassify import TopographicClassifier, quick_classify
from topoclassify.utils import plot_results, save_geotiff, get_class_stats

## 1. Quick Classification (1 baris kode)

In [ ]:
# Ganti dengan path DEM Anda
dem_path = "path/to/your/dem.tif"

# Klasifikasi cepat
result = quick_classify(dem_path, output_path="classification_result.tif")

## 2. Step-by-Step Classification

In [ ]:
# Inisialisasi classifier
classifier = TopographicClassifier(dem_path)

# Info basic DEM
print(f"DEM shape: {classifier.dem_data.shape}")
print(f"DEM elevation range: {np.nanmin(classifier.dem_data):.1f} - {np.nanmax(classifier.dem_data):.1f} m")
print(f"DEM coordinate system: {classifier.profile['crs']}")

In [ ]:
# Hitung metrik topografi secara terpisah
slope = classifier.slope
convexity = classifier.convexity
texture = classifier.texture

print("Metrics calculated successfully!")
print(f"Slope range: {np.nanmin(slope):.2f} - {np.nanmax(slope):.2f} degrees")
print(f"Convexity range: {np.nanmin(convexity):.3f} - {np.nanmax(convexity):.3f}")
print(f"Texture range: {np.nanmin(texture):.3f} - {np.nanmax(texture):.3f}")

In [ ]:
# Jalankan klasifikasi penuh (24 kelas)
classification = classifier.classify()

## 3. Visualisasi Hasil

In [ ]:
# Plot hasil klasifikasi dan metrik
plot_results(classification, slope, convexity, texture)

In [ ]:
# Plot distribusi kelas
unique_classes, counts = np.unique(classification[classification > 0], return_counts=True)

plt.figure(figsize=(15, 5))
plt.bar(unique_classes, counts, color='steelblue', alpha=0.7)
plt.xlabel('Topographic Class')
plt.ylabel('Number of Pixels')
plt.title('Distribution of Topographic Classes')
plt.xticks(unique_classes)
plt.grid(True, alpha=0.3)
plt.show()

print(f"Total classes found: {len(unique_classes)}")
print(f"Classes: {list(unique_classes)}")

## 4. Analisis Statistik

In [ ]:
# Dapatkan statistik per kelas
stats = get_class_stats(classification, slope, convexity, texture)

# Tampilkan dalam format tabel
print("Class Statistics:")
print("-" * 85)
print(f"{'Class':<6} {'Pixels':<8} {'Slope (°)':<10} {'Convexity':<12} {'Texture':<10} {'Area %':<8}")
print("-" * 85)

total_pixels = np.sum([s['pixel_count'] for s in stats.values()])

for class_id in sorted(stats.keys()):
    stat = stats[class_id]
    area_pct = (stat['pixel_count'] / total_pixels) * 100
    print(f"{class_id:<6} {stat['pixel_count']:<8} {stat['mean_slope']:<10.2f} "
          f"{stat['mean_convexity']:<12.3f} {stat['mean_texture']:<10.3f} {area_pct:<8.2f}")

In [ ]:
# Dapatkan deskripsi kelas
descriptions = classifier.get_class_descriptions()

print("\nClass Descriptions (Iwahashi & Pike 2007):")
print("=" * 60)
for class_id in sorted(unique_classes):
    if class_id in descriptions and class_id in stats:
        pixel_count = stats[class_id]['pixel_count']
        area_pct = (pixel_count / total_pixels) * 100
        print(f"Class {class_id:2d} ({area_pct:5.1f}%): {descriptions[class_id]}")

## 5. Simpan Hasil

In [ ]:
# Simpan klasifikasi sebagai GeoTIFF
save_geotiff(classification, "topographic_classification_24class.tif", classifier.profile)

# Simpan metrik individual juga
save_geotiff((slope * 100).astype(np.uint16), "slope_degrees_x100.tif", classifier.profile)
save_geotiff((convexity * 1000).astype(np.uint16), "convexity_x1000.tif", classifier.profile)
save_geotiff((texture * 1000).astype(np.uint16), "texture_x1000.tif", classifier.profile)

print("All results saved successfully!")

## 6. Export Statistik ke CSV

In [ ]:
import pandas as pd

# Buat DataFrame dari statistik
stats_data = []
for class_id in sorted(stats.keys()):
    stat = stats[class_id]
    area_pct = (stat['pixel_count'] / total_pixels) * 100
    
    stats_data.append({
        'Class': class_id,
        'Description': descriptions.get(class_id, 'Unknown'),
        'Pixel_Count': stat['pixel_count'],
        'Area_Percent': area_pct,
        'Mean_Slope': stat['mean_slope'],
        'Mean_Convexity': stat['mean_convexity'],
        'Mean_Texture': stat['mean_texture']
    })

df = pd.DataFrame(stats_data)
df.to_csv('topographic_classification_stats.csv', index=False)

print("Statistics exported to CSV!")
df.head()